# Паттерн Команда

-поведенческий паттерн проектирования

-превращает запросы в объекты

-позволяет передавать эти объекты как аргументы при вызове методов

-ставить запросы в очередь, логировать их

-также поддерживать отмену операций.

In [1]:
from __future__ import annotations
from abc import ABC, abstractmethod

In [2]:
class BaseCommand(ABC):
    """Инетрфейс команд -  
    Абстрактный класс команды, имеющий единственный абстрактный метод, обязательный для конкретных команд"""
        
    @abstractmethod
    def execute(self, context):
        """Метод, выполняющий команду. Будет переопределяться для каждой команды"""
        pass

In [3]:
### Действующие лица

class VCS:
    """Класс 'Отправитель' - Система контроля версий"""
    
    def __init__(self,name):
        self.name = name
    
    def __str__(self):
        return self.name
    
    """Любой запрос"""
    def invoke(self, command: BaseCommand, data):
        print(f'{self.name}: {command}')
        command.execute(data)

class Repo:
    """Класс 'Получатель' - Удаленный репозиторий"""
    history=[]
    
    def __init__(self, name):
        self.name = name
        
    def __str__(self):
        return self.name
        
    def push(self, new_version):
        self.history.append(f'push {new_version}')
        print(f'{self.name}: Изменения сохранены')
        
    def pull(self, branch):
        self.history.append(f'pull {branch}')
        print(f'{self.name}: Пулл реквест из {branch}')
        
    def commit(self, message):
        self.history.append(f'commit {message}')
        print(f'{self.name}: Создан коммит - "{message}"')
        
    def reset(self, commit_hash):
        self.history.pop()
        print(f'{self.name}: Откат версии до {commit_hash}')
    
    def log(self):
        print(f"История изменений: {self.history}")

In [4]:
#Набор команд

class PushCommand(BaseCommand):
    
    def __init__(self, reciever: Repo):
        self.reciever = reciever
        
    def execute(self, smth_new):
        print(f'push {smth_new}')
        self.reciever.push(smth_new)
        

class PullCommand(BaseCommand):
    
    def __init__(self, reciever: Repo):
        self.reciever = reciever
        
        
        
    def execute(self, branch_name):
        print(f'pull {branch_name}')
        self.reciever.pull(branch_name)
        
class CommitCommand(BaseCommand):
    
    def __init__(self, reciever: Repo):
        self.reciever = reciever
        
    def execute(self, text):
        print(f'commit -m {text}')
        self.reciever.commit(text)
        
class ResetCommand(BaseCommand):
    
    def __init__(self, reciever: Repo):
        self.reciever = reciever
        
    def execute(self, version):
        print(f'reset {version}')
        self.reciever.reset(version)
        
class LogCommand(BaseCommand):
    
    def __init__(self, reciever: Repo):
        self.reciever = reciever
        
    def execute(self, t):
        print(f'log {t}')
        self.reciever.log()

___

In [5]:
# Два отправителя
cmd = VCS('Cmd')
gui = VCS('Gui')

In [6]:
# Получатель
Unifloc = Repo('Unifloc')

In [7]:
#Набор команд
pull = PullCommand(reciever=Unifloc)
push = PushCommand(reciever=Unifloc)
commit = CommitCommand(reciever=Unifloc)
reset = ResetCommand(reciever=Unifloc)
log = LogCommand(reciever=Unifloc)

In [8]:
#Реализация команд
cmd.invoke(pull,'master')

Cmd: <__main__.PullCommand object at 0x7ffdf1f63ee0>
pull master
Unifloc: Пулл реквест из master


In [9]:
cmd.invoke(commit, 'Новая фича')

Cmd: <__main__.CommitCommand object at 0x7ffdf1f63b50>
commit -m Новая фича
Unifloc: Создан коммит - "Новая фича"


In [10]:
gui.invoke(push, 'master')

Gui: <__main__.PushCommand object at 0x7ffdf1f63460>
push master
Unifloc: Изменения сохранены


In [11]:
cmd.invoke(reset, '23d5tg')

Cmd: <__main__.ResetCommand object at 0x7ffdf1f63e50>
reset 23d5tg
Unifloc: Откат версии до 23d5tg


In [12]:
cmd.invoke(log, '--oneline')

Cmd: <__main__.LogCommand object at 0x7ffdf1f637c0>
log --oneline
История изменений: ['pull master', 'commit Новая фича']


In [13]:
cmd.invoke(pull,'master')

Cmd: <__main__.PullCommand object at 0x7ffdf1f63ee0>
pull master
Unifloc: Пулл реквест из master


In [14]:
cmd.invoke(log, '--oneline')

Cmd: <__main__.LogCommand object at 0x7ffdf1f637c0>
log --oneline
История изменений: ['pull master', 'commit Новая фича', 'pull master']
